# transmart python client with JupyterLab integration
A short demonstration on getting data from tranSMART into the Jupyter Notebook analytical environment.

In [1]:
import transmart as tm
from transmart.api.v2.dashboard import Dashboard

## Getting a user account

Before being able to connect to our demo environment, make sure you have account in our user management system. You are able to self register.

Link to create account: [KeyCloak registration](https://keycloak-dwh-test.thehyve.net/auth/realms/transmart/protocol/openid-connect/auth?response_type=code&client_id=transmart-client)

If you are unable to create an account, you can use the dummy account and password: ``demo-user`` 

# User account details
You can (optionally) fill in you user details here. 

In [4]:
username = None
password = None  # Can be None, your KeyCloak password will be asked for.

First create a connection with a transmart instance with V2 api enabled. This could take a litlle time as some caches are built.

In [2]:
api = tm.get_api(
    host = 'https://transmart.thehyve.net',
    kc_url = "https://keycloak-dwh-test.thehyve.net",
    kc_realm = "transmart",
    user=username, 
    password=password, 
    print_urls=True
)

https://transmart.thehyve.net/v2/studies
https://transmart.thehyve.net/v2/tree_nodes?depth=0&counts=False&tags=True
No valid cache found. Building indexes...
Finished in 1.10 seconds
https://transmart.thehyve.net/v2/pedigree/relation_types


The main objects to query transmart are created here. This `ObservationConstraint` object can be used specified and combined to create queries to the api.

In [3]:
c = api.new_constraint(study='ORACLE_1000_PATIENT', concept='O1KP:NUM39')
c

ObservationConstraint(concept='O1KP:NUM39', study='ORACLE_1000_PATIENT')

An example call that gets the counts for the chosen constraints

In [4]:
c.observations.counts()

https://transmart.thehyve.net/v2/observations/counts


{'observationCount': 1200, 'patientCount': 1200}

Further specify the constraint using intuitive attributes.

In [5]:
c.min_value = 15
c

ObservationConstraint(concept='O1KP:NUM39', min_value=15, study='ORACLE_1000_PATIENT')

In [6]:
c.observations.counts()

https://transmart.thehyve.net/v2/observations/counts


{'observationCount': 202, 'patientCount': 202}

In [7]:
print('Possible attributes to specify constraints:\n- ', end='')
print('\n- '.join(sorted(c.params.keys())))

Possible attributes to specify constraints:
- concept
- max_date_value
- max_start_date
- max_value
- min_date_value
- min_start_date
- min_value
- study
- subject_set_id
- trial_visit
- value_list


In [8]:
api.observations.aggregates_per_concept(c)

https://transmart.thehyve.net/v2/observations/aggregates_per_concept


{'aggregatesPerConcept': {'O1KP:NUM39': {'numericalValueAggregates': {'avg': 17.60090668316832,
    'count': 202,
    'max': 27.10521,
    'min': 15.01558,
    'stdDev': 2.4378601422759876}}}}

In [9]:
obs = api.observations(c)

https://transmart.thehyve.net/v2/observations?constraint={"args": [{"conceptCode": "O1KP:NUM39", "type": "concept"}, {"operator": ">=", "value": 15, "type": "value", "valueType": "NUMERIC"}, {"studyId": "ORACLE_1000_PATIENT", "type": "study_name"}], "type": "and"}&type=clinical


In [10]:
obs.dataframe.head()

concept.conceptCode                                concept.conceptPath  \
0          O1KP:NUM39  \Public Studies\Oracle_1000_Patient\Numerical ...   
1          O1KP:NUM39  \Public Studies\Oracle_1000_Patient\Numerical ...   
2          O1KP:NUM39  \Public Studies\Oracle_1000_Patient\Numerical ...   
3          O1KP:NUM39  \Public Studies\Oracle_1000_Patient\Numerical ...   
4          O1KP:NUM39  \Public Studies\Oracle_1000_Patient\Numerical ...   

   concept.name  numericValue  patient.age patient.birthDate  \
0  numerical_39      16.13731           50              None   
1  numerical_39      17.30936           50              None   
2  numerical_39      17.69275           50              None   
3  numerical_39      15.57817           50              None   
4  numerical_39      16.41986           50              None   

  patient.deathDate  patient.id patient.inTrialId patient.maritalStatus  \
0              None       -1475       subject_475                  None   
1              None       -1714       subject_714                  None   
2              None       -1253       subject_253                  None   
3              None       -1787       subject_787                  None   
4              None       -1827       subject_827                  None   

  patient.race patient.religion patient.sex patient.sexCd  \
0         None             None        male          Male   
1         None             None        male          Male   
2         None             None        male          Male   
3         None             None        male          Male   
4         None             None      female        Female   

         patient.trial           study.name  
0  ORACLE_1000_PATIENT  ORACLE_1000_PATIENT  
1  ORACLE_1000_PATIENT  ORACLE_1000_PATIENT  
2  ORACLE_1000_PATIENT  ORACLE_1000_PATIENT  
3  ORACLE_1000_PATIENT  ORACLE_1000_PATIENT  
4  ORACLE_1000_PATIENT  ORACLE_1000_PATIENT

## Integration with widgets and bqplot

Methods `find_concept()` and `interact()` can be used to visually create and modify a constraint object.

In [ ]:
c.find_concept('study:oracle')

In [ ]:
c.interact()

And there is dashboard for exploration.

In [ ]:
dash = Dashboard(api, patients=c)

In [ ]:
dash.get()